In [1]:
from src.datasets import BasePostsDataset, BaseFactCheckDataset, TextConcatPosts
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

tasks_path = "data/complete_data/tasks.json"
posts_path = "data/complete_data/posts.csv"
fact_checks_path = "data/complete_data/fact_checks.csv"
gs_path = "data/complete_data/pairs.csv"


posts = TextConcatPosts(posts_path, tasks_path, task_name="monolingual", gs_path=gs_path, lang="spa")

posts.df_dev


,ocr,verdicts,text,fb,tw,ig,full_text,gs
post_id,,,,,,,,
48,"""Un pueblo que elige corruptos, impostores, la...",Partly false information,,1,0,0,"""Un pueblo que elige corruptos, impostores, la...",[]
149,10:39 PM | 8.9kB/s ← Tweet Euquico [USER] f Gr...,Partly false information,,1,0,0,10:39 PM | 8.9kB/s ← Tweet Euquico [USER] f Gr...,[]
280,"44 ANDRÉS FELIPE ARIAS NO SE HA FUGADO, ESTA E...",,,1,0,0,"44 ANDRÉS FELIPE ARIAS NO SE HA FUGADO, ESTA E...",[]
324,"8:26 PM 3 Resistencia Uruguay 154 miembros, 3 ...",False information,,1,0,0,"8:26 PM 3 Resistencia Uruguay 154 miembros, 3 ...",[]
326,95 CO DE OME BANCO DEL COLOM 50% SALARIO MÍNIM...,Partly false information,,1,0,0,95 CO DE OME BANCO DEL COLOM 50% SALARIO MÍNIM...,[]
...,...,...,...,...,...,...,...,...
27936,,False information,🛑🛑🛑🛑🛑🛑🛑Así esta azotando Nicaragua el Huracán ...,1,0,0,🛑🛑🛑🛑🛑🛑🛑Así esta azotando Nicaragua el Huracán...,[]
27946,,Missing context,🟢 OREMOS POR UCRANIA. Los cristianos ucraniano...,1,0,0,🟢 OREMOS POR UCRANIA. Los cristianos ucranian...,[]
27984,SUBTENIENT EVENEGAS K 1995,False information,"🤔 Mientras que Trump tilda de ""matones"" a mani...",1,0,0,SUBTENIENT EVENEGAS K 1995 🤔 Mientras que Trum...,[]


In [1]:
from src.datasets import BasePostsDataset, BaseFactCheckDataset
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

tasks_path = "data/complete_data/tasks.json"
posts_path = "data/complete_data/posts.csv"
fact_checks_path = "data/complete_data/fact_checks.csv"
gs_path = "data/complete_data/pairs.csv"

posts = BasePostsDataset(posts_path, tasks_path, task_name="monolingual", gs_path=gs_path, lang="spa")
fact_checks = BaseFactCheckDataset(fact_checks_path, tasks_path, task_name="monolingual", lang="spa")

df_fc = fact_checks.df
df_posts_train = posts.df_train
df_posts_dev = posts.df_dev

In [2]:
df_fc.head()

,claim,instances,title
fact_check_id,,,
56,"""(A Pere Aragonés) ustedes eliminaron la conse...",[https://www.newtral.es/fact-check-medio-ambie...,El Departamento de Medio Ambiente de Cataluña ...
58,"""(Andalucía tiene) un 36% de paro juvenil de m...",[https://www.newtral.es/paro-juvenil-andalucia...,Andalucía no dobla al resto de España en paro ...
59,"""(Biden) destruirá la protección para las enfe...",[https://www.telemundo.com/noticias/noticias-t...,"No, Biden no eliminaría la cobertura de salud ..."
64,"""(El PP) vincula su oposición a renovar el CGP...",[https://www.newtral.es/factcheck-abalos-pp-re...,El PP no ha vinculado su oposición a renovar e...
65,"""(En Euskadi) llevamos 40 años de victorias el...",[https://www.newtral.es/el-bloque-de-izquierda...,El bloque de izquierda no lleva 40 años de vic...


In [3]:
df_posts_train.head()

,ocr,verdicts,text,fb,tw,ig,gs
post_id,,,,,,,
4,"""Bienaventurados los perseguidos por mi causa ...",,,1,0,0,[80729]
6,"""Coman lo que quieran en Semana Santa, el sacr...",False information,,1,0,0,[50769]
7,"""DISCURSO DE PEDRO CASTILLO, ESTUBO BASADO EN ...",False information,,1,0,0,[56968]
8,"""Debemos ser Solidarios con los que menos tien...",Partly false information,,1,0,0,[148668]
11,"""EL AGUINALDO DE 2 MILLONES NO ES SOLO PARA NO...",False information,,1,0,0,"[3332, 51854]"


In [4]:
df_fc["full_text"] = df_fc["title"] + " " + df_fc["claim"]
df_posts_train["full_text"] = df_posts_train["ocr"] + " " + df_posts_train["text"]
df_posts_dev["full_text"] = df_posts_dev["ocr"] + " " + df_posts_dev["text"]

df_fc["full_text"] = df_fc["full_text"].str.lower()
df_posts_train["full_text"] = df_posts_train["full_text"].str.lower()
df_posts_dev["full_text"] = df_posts_dev["full_text"].str.lower()


In [5]:
import torch
from sentence_transformers import SentenceTransformer

# Load the model and use GPU
print("Load Model")
model = SentenceTransformer('/home/bsc/bsc830651/.cache/huggingface/hub/models--intfloat--multilingual-e5-large/snapshots/ab10c1a7f42e74530fe7ae5be82e6d4f11a719eb', device="cuda")
# model = SentenceTransformer('/home/bsc/bsc830651/.cache/huggingface/hub/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/79f2382ceacceacdf38563d7c5d16b9ff8d725d6', device="cuda")

# Encode the posts and fact-checks as PyTorch tensors directly on GPU
print("Encode Posts")
emb_post = torch.tensor(model.encode(df_posts_train["full_text"].values, device="cuda", show_progress_bar=True, batch_size=128, normalize_embeddings=True))
emb_post_dev = torch.tensor(model.encode(df_posts_dev["full_text"].values, device="cuda", show_progress_bar=True, batch_size=128, normalize_embeddings=True))

print("Encode Fact Check pool")
emb_fc = torch.tensor(model.encode(df_fc["full_text"].values, device="cuda", show_progress_bar=True, batch_size=128, normalize_embeddings=True))

# Compute cosine similarity using matrix multiplication on GPU
print("Compute similarity")
prec_similarity = torch.mm(emb_post, emb_fc.T).cpu().numpy()
prec_similarity_dev = torch.mm(emb_post_dev, emb_fc.T).cpu().numpy()


/home/bsc/bsc830651/.conda/envs/factcheck/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Load Model
Encode Posts


Batches: 100%|██████████| 5/5 [00:01<00:00,  2.63it/s]


Encode Fact Check pool


Batches: 100%|██████████| 111/111 [00:15<00:00,  7.38it/s]


Compute similarity


AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [13]:
def get_k_similar_posts(emb_post, emb_fc, k=10,):
    # emb_post_sparse = sparse.csr_matrix(post)
    # emb_fc_sparse = sparse.csr_matrix(fc_pool)

    similarity = torch.mm(emb_post.reshape(1, -1), emb_fc.T).cpu().numpy()

    # Sort in descending order
    st_similarity = np.sort(similarity)[::-1]
    idx_sim = np.argsort(similarity)[::-1]
    print(st_similarity[:k],"\n", idx_sim[:k])
    return idx_sim[:k], st_similarity[:k]

def compute_similar_posts(post_ids, df_posts, df_fc, similarity, k=10, show=False):

    idx_to_pos = {idx: pos for pos, idx in enumerate(df_posts.index)}
    pos_to_idx_fc = {pos: idx for pos, idx in enumerate(df_fc.index)}

    ls_pos = [idx_to_pos[post_id] for post_id in post_ids]
    idx_sim, similarity = np.argsort(similarity[ls_pos, :], axis=1)[:, ::-1][:, :k], np.sort(similarity[ls_pos, :], axis=1)[:, ::-1][:, :k]

    # Apply the function element-wise to the array
    vectorized_map = np.vectorize(lambda x: pos_to_idx_fc.get(x, None))
    fc_ids = vectorized_map(idx_sim)

    if show:
    # Show the full text of the post
        print(df_posts.loc[post_ids, :].to_markdown())
        print("="*100)
        print("\n\n")
        for i, idx in enumerate(idx_sim):
            print(f"Similarity: {similarity[i]}")
            print(df_fc.iloc[idx].to_markdown())
            print("\n\n")
            
    return fc_ids, similarity


compute_similar_posts([11, 8], df_posts_train, df_fc, similarity=prec_similarity, show=True);

|   post_id | ocr                                                                                                                                                                                                                               | verdicts                 | text   |   fb |   tw |   ig | gs            | full_text                                                                                                                                                                                                                         | preds                                                                         |
|----------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------|:-------|-----:|-----:|-----:|:--------------|:--------------------------------------------------------------

In [8]:
df_posts_train["preds"] = compute_similar_posts(df_posts_train.index, df_posts_train, df_fc, similarity=prec_similarity, k=10)[0].tolist()
print(df_posts_train.apply(lambda x: len(list(set(x["gs"]).intersection(set(x["preds"])))) > 0, axis=1).mean())
df_posts_train.head()

0.8930348258706468


,ocr,verdicts,text,fb,tw,ig,gs,full_text,preds
post_id,,,,,,,,,
4,"""Bienaventurados los perseguidos por mi causa ...",,,1,0,0,[80729],"""bienaventurados los perseguidos por mi causa ...","[80729, 82043, 81982, 53979, 89563, 81835, 107..."
6,"""Coman lo que quieran en Semana Santa, el sacr...",False information,,1,0,0,[50769],"""coman lo que quieran en semana santa, el sacr...","[50769, 80782, 101111, 39736, 27415, 36525, 90..."
7,"""DISCURSO DE PEDRO CASTILLO, ESTUBO BASADO EN ...",False information,,1,0,0,[56968],"""discurso de pedro castillo, estubo basado en ...","[56968, 152115, 38000, 101078, 107780, 35886, ..."
8,"""Debemos ser Solidarios con los que menos tien...",Partly false information,,1,0,0,[148668],"""debemos ser solidarios con los que menos tien...","[148668, 197461, 102321, 102317, 102313, 10231..."
11,"""EL AGUINALDO DE 2 MILLONES NO ES SOLO PARA NO...",False information,,1,0,0,"[3332, 51854]","""el aguinaldo de 2 millones no es solo para no...","[51854, 3332, 195763, 37266, 6070, 55226, 8479..."


In [15]:
df_posts_dev["preds"] = compute_similar_posts(df_posts_dev.index, df_posts_dev, df_fc, similarity=prec_similarity_dev, k=10)[0].tolist()
print(df_posts_dev.apply(lambda x: len(list(set(x["gs"]).intersection(set(x["preds"])))) > 0, axis=1).mean())
df_posts_dev.head()

0.0


,ocr,verdicts,text,fb,tw,ig,gs,full_text,preds
post_id,,,,,,,,,
48,"""Un pueblo que elige corruptos, impostores, la...",Partly false information,,1,0,0,[],"""un pueblo que elige corruptos, impostores, la...","[98618, 20499, 20498, 54015, 35924, 53032, 154..."
149,10:39 PM | 8.9kB/s ← Tweet Euquico [USER] f Gr...,Partly false information,,1,0,0,[],10:39 pm | 8.9kb/s ← tweet euquico [user] f gr...,"[39355, 50905, 89359, 137496, 48291, 74222, 50..."
280,"44 ANDRÉS FELIPE ARIAS NO SE HA FUGADO, ESTA E...",,,1,0,0,[],"44 andrés felipe arias no se ha fugado, esta e...","[74210, 150855, 112997, 113641, 140952, 118496..."
324,"8:26 PM 3 Resistencia Uruguay 154 miembros, 3 ...",False information,,1,0,0,[],"8:26 pm 3 resistencia uruguay 154 miembros, 3 ...","[80872, 80873, 52933, 80871, 82062, 84432, 908..."
326,95 CO DE OME BANCO DEL COLOM 50% SALARIO MÍNIM...,Partly false information,,1,0,0,[],95 co de ome banco del colom 50% salario mínim...,"[51436, 51433, 51434, 115426, 195833, 51197, 1..."


In [14]:
compute_similar_posts([48, 149], df_posts_dev, df_fc, similarity=prec_similarity_dev, show=True);

|   post_id | ocr                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | verdicts                 | text   |   fb |   tw |   ig | gs   | full_text                                                                                                                                             

In [25]:
np.dot(emb_post[0], emb_fc[0]) / (np.linalg.norm(emb_post[0]) * np.linalg.norm(emb_fc[0]))

np.float32(0.24843584)

In [27]:
np.dot(emb_post[0], emb_fc[1]) / (np.linalg.norm(emb_post[0]) * np.linalg.norm(emb_fc[1]))

np.float32(0.20492055)

In [28]:
np.dot(emb_post[1], emb_fc[0]) / (np.linalg.norm(emb_post[1]) * np.linalg.norm(emb_fc[0]))

np.float32(0.3132233)